In [1]:
import os
import datetime
import time

import IPython
import IPython.display
from ipywidgets import IntProgress
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

from sklearn.metrics import r2_score
import math

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

C:\Users\iasha\AppData\Local\Temp\ipykernel_15924\424092037.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def convert_line_to_list(line: str):
    split_line = line.split(' ')
    result_list = []

    for current in split_line:
        if current == '\n':
            continue
            
        result_list.append(int(current))

    return result_list

In [4]:
def get_mlt(clustering_unit_index):
    return clustering_unit_index // 80 * 0.25

def get_magneticLat(clustering_unit_index):
    return clustering_unit_index % 80 * 0.5 + 50

In [5]:
def cluster_unit_index_to_file_path(clustering_unit_index):
    mlt = get_mlt(clustering_unit_index)
    magnetic_lat = get_magneticLat(clustering_unit_index)
    
    return f"C:\\Coding\\final-qualifying-work_2023-2024\\OvationPrimeDataCollector\\north-nowcast-diffuse\\{mlt:.2f}_{magnetic_lat:.1f}.csv"

In [6]:
def get_cluster_centre(cluster):
    cluster_centre = pd.read_csv(cluster_unit_index_to_file_path(cluster[0]), sep=';', low_memory=False)

    f = IntProgress(min=0, max=len(cluster))
    display(f)
    i = 0
    for clustering_unit_index in cluster:
        # if i % 100 == 0:
        #     f.value += 100
        f.value += 1
        if i == 0:
            i += 1
            continue
        current_df = pd.read_csv(cluster_unit_index_to_file_path(clustering_unit_index), sep=';', low_memory=False)
        cluster_centre['y'] += current_df['y']
        i += 1

    cluster_centre['y'] = cluster_centre['y'].divide(len(cluster))
    return cluster_centre

In [7]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [8]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [9]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [10]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [11]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')
    
  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [25]:
def investigate(cluster_index, model, conv_width, clusters, save_data):
    cluster_centre = get_cluster_centre(clusters[cluster_index])
    cluster_centre = cluster_centre.drop(['date'], axis=1)
    cluster_centre = cluster_centre.drop(['segment'], axis=1)
    
    n = len(cluster_centre)
    train_df = cluster_centre[0:int(n*0.7)]
    val_df = cluster_centre[int(n*0.7):int(n*0.9)]
    test_df = cluster_centre[int(n*0.9):]

    train_mean = train_df.mean()
    train_std = train_df.std()

    train_df = (train_df - train_mean) / train_std
    val_df = (val_df - train_mean) / train_std
    test_df = (test_df - train_mean) / train_std
    
    conv_window = WindowGenerator(
    input_width=conv_width,
    label_width=1,
    shift=1,
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    label_columns=['y']
    )

    model = keras.models.Sequential.from_config(model.get_config())
    
    history = compile_and_fit(model, conv_window)

    inputs, labels = conv_window.example

    val_df_numpy = val_df.to_numpy()
    train_df_numpy = train_df.to_numpy()
    test_df_numpy = test_df.to_numpy()
    new_true_values_val = []
    new_true_values_train = []
    new_true_values_test = []
    new_input_val = []
    new_input_train = []
    new_input_test = []

    new_true_values_val = val_df_numpy[3:].flatten()
    new_input_val = np.column_stack([val_df_numpy[0:-3], val_df_numpy[1:-2], val_df_numpy[2:-1]])

    new_true_values_train = train_df_numpy[3:].flatten()
    new_input_train = np.column_stack([train_df_numpy[0:-3], train_df_numpy[1:-2], train_df_numpy[2:-1]])

    new_true_values_test = test_df_numpy[3:].flatten()
    new_input_test = np.column_stack([test_df_numpy[0:-3], test_df_numpy[1:-2], test_df_numpy[2:-1]])
        
    val_predictions = model(np.array(new_input_val)).numpy()
    train_predictions = model(np.array(new_input_train)).numpy()
    test_predictions = model(np.array(new_input_test)).numpy()
    
    new_val_predictions = val_predictions.flatten()
    new_train_predictions = train_predictions.flatten()
    new_test_predictions = test_predictions.flatten()
    
    new_true_values_val = (new_true_values_val * train_std['y']) + train_mean['y']
    new_val_predictions = (new_val_predictions * train_std['y']) + train_mean['y']

    new_true_values_train = (new_true_values_train * train_std['y']) + train_mean['y']
    new_train_predictions = (new_train_predictions * train_std['y']) + train_mean['y']

    new_true_values_test = (new_true_values_test * train_std['y']) + train_mean['y']
    new_test_predictions = (new_test_predictions * train_std['y']) + train_mean['y']

    print("***TRAIN***:")
    output_file.write("***TRAIN***:\n")
    print(f"correlation related to cluster centre: {np.corrcoef(new_true_values_train, new_train_predictions)[0][1]:.2f}")
    output_file.write(f"correlation related to cluster centre: {np.corrcoef(new_true_values_train, new_train_predictions)[0][1]:.2f}\n")
    print(f"r2 score related to cluster centre: {r2_score(new_true_values_train, new_train_predictions):.2f}")
    output_file.write(f"r2 score related to cluster centre: {r2_score(new_true_values_train, new_train_predictions):.2f}\n")
    print("***VALIDATION***:")
    output_file.write("***VALIDATION***:\n")
    print(f"correlation related to cluster centre: {np.corrcoef(new_true_values_val, new_val_predictions)[0][1]:.2f}")
    output_file.write(f"correlation related to cluster centre: {np.corrcoef(new_true_values_val, new_val_predictions)[0][1]:.2f}\n")
    print(f"r2 score related to cluster centre: {r2_score(new_true_values_val, new_val_predictions):.2f}")
    output_file.write(f"r2 score related to cluster centre: {r2_score(new_true_values_val, new_val_predictions):.2f}\n")
    print("***TEST***:")
    output_file.write("***TEST***:\n")
    print(f"correlation related to cluster centre: {np.corrcoef(new_true_values_test, new_test_predictions)[0][1]:.2f}")
    output_file.write(f"correlation related to cluster centre: {np.corrcoef(new_true_values_test, new_test_predictions)[0][1]:.2f}\n")
    print(f"r2 score related to cluster centre: {r2_score(new_true_values_test, new_test_predictions):.2f}")
    output_file.write(f"r2 score related to cluster centre: {r2_score(new_true_values_test, new_test_predictions):.2f}\n")

    sum_correlation_val = 0
    sum_r2_val = 0

    sum_correlation_test = 0
    sum_r2_test = 0

    f = IntProgress(min=0, max=len(clusters[cluster_index]))
    display(f)

    i = 0
    for clustering_unit_index in clusters[cluster_index]:
        if (i % 100 == 0):
            f.value += 100
        
        current_df = pd.read_csv(cluster_unit_index_to_file_path(clustering_unit_index), sep=';', low_memory=False)
        current_df = current_df.drop(['date'], axis=1)
        current_df = current_df.drop(['segment'], axis=1)

        n = len(current_df)
        current_train_df = current_df[0:int(n*0.7)]
        current_val_df = current_df[int(n*0.7):int(n*0.9)]
        current_test_df = current_df[int(n*0.9):]

        current_train_df = (current_train_df - train_mean) / train_std
        current_val_df = (current_val_df - train_mean) / train_std
        current_test_df = (current_test_df - train_mean) / train_std

        current_val_df_numpy = current_val_df.to_numpy()
        current_new_true_values_val = current_val_df_numpy[3:].flatten()
        current_new_input_val = np.column_stack([current_val_df_numpy[0:-3], current_val_df_numpy[1:-2], current_val_df_numpy[2:-1]])

        current_test_df_numpy = current_test_df.to_numpy()
        current_new_true_values_test = current_test_df_numpy[3:].flatten()
        current_new_input_test = np.column_stack([current_test_df_numpy[0:-3], current_test_df_numpy[1:-2], current_test_df_numpy[2:-1]])

        current_val_predictions = model(np.array(current_new_input_val)).numpy()
        current_new_val_predictions = current_val_predictions.flatten()

        current_test_predictions = model(np.array(current_new_input_test)).numpy()
        current_new_test_predictions = current_test_predictions.flatten()

        current_new_true_values_val = (current_new_true_values_val * train_std['y']) + train_mean['y']
        current_new_val_predictions = (current_new_val_predictions * train_std['y']) + train_mean['y']

        current_new_true_values_test = (current_new_true_values_test * train_std['y']) + train_mean['y']
        current_new_test_predictions = (current_new_test_predictions * train_std['y']) + train_mean['y']
            
        sum_correlation_val += np.corrcoef(current_new_true_values_val, current_new_val_predictions)[0][1]
        sum_r2_val += r2_score(current_new_true_values_val, current_new_val_predictions)

        sum_correlation_test += np.corrcoef(current_new_true_values_test, current_new_test_predictions)[0][1]
        sum_r2_test += r2_score(current_new_true_values_test, current_new_test_predictions)

        i += 1

    if (save_data):
        f = open(f"out\\train-mean-and-std\\train mean and std for cluster {cluster_index}.txt", "w")
        f.write(str(train_mean.mean()) + ' ' + str(train_std.mean()))
        f.close()
        model.save(f"out\\saved-models\\model for cluster {cluster_index}")

    cluster_len = len(clusters[cluster_index])
    
    return sum_correlation_val / cluster_len, sum_r2_val / cluster_len, sum_correlation_test / cluster_len, sum_r2_test / cluster_len

In [37]:
def do_investigations(model, clusters, conv_width):
    cluster_sizes = []
    average_correlations_val = []
    average_r2_scores_val = []
    average_correlations_test = []
    average_r2_scores_test = []

    for i in range(len(clusters)):
        print(f"---do_investigations. cluster index : {i}---")
        output_file.write(f"---do_investigations. cluster index : {i}---\n")
        average_correlation_val, average_r2_val, average_correlation_test, average_r2_test = investigate(i, model, conv_width, clusters, True)
        
        print("***VALIDATION***:")
        output_file.write("***VALIDATION***:\n")
        print(f"average correlation val: {average_correlation_val:.2f}")
        output_file.write(f"average correlation val: {average_correlation_val:.2f}\n")
        print(f"average r2 val: {average_r2_val:.2f}")
        output_file.write(f"average r2 val: {average_r2_val:.2f}\n")
        print("***TEST***:")
        output_file.write("***TEST***:\n")
        print(f"average correlation test: {average_correlation_test:.2f}")
        output_file.write(f"average correlation test: {average_correlation_test:.2f}\n")
        print(f"average r2 test: {average_r2_test:.2f}")
        output_file.write(f"average r2 test: {average_r2_test:.2f}\n")

        cluster_sizes.append(len(clusters[i]))
        average_correlations_val.append(average_correlation_val)
        average_r2_scores_val.append(average_r2_val)
        average_correlations_test.append(average_correlation_test)
        average_r2_scores_test.append(average_r2_test)

    return cluster_sizes, average_correlations_val, average_r2_scores_val, average_correlations_test, average_r2_scores_test

In [35]:
def print_metrics(cluster_sizes, average_correlations_val, average_r2_scores_val, average_correlations_test, average_r2_scores_test, clusters):
    total_size = 0
    correlation_sum_val = 0
    r2_score_sum_val = 0
    correlation_sum_test = 0
    r2_score_sum_test = 0
    
    bad_clusters_indices = []

    for i in range(len(cluster_sizes)):
        if (math.isnan(average_correlations_val[i]) or (average_r2_scores_val[i] < 0)):
            bad_clusters_indices.append(i)
            continue
        
        correlation_sum_val += cluster_sizes[i] * average_correlations_val[i]
        r2_score_sum_val += cluster_sizes[i] * average_r2_scores_val[i]
        correlation_sum_test += cluster_sizes[i] * average_correlations_test[i]
        r2_score_sum_test += cluster_sizes[i] * average_r2_scores_test[i]
        total_size += cluster_sizes[i]

    weighted_average_correlation_val = correlation_sum_val / total_size
    weighted_average_r2_score_val = r2_score_sum_val / total_size
    weighted_average_correlation_test = correlation_sum_test / total_size
    weighted_average_r2_score_test = r2_score_sum_test / total_size

    print(f"weighted_average_correlation_val: {weighted_average_correlation_val:.2f}")
    output_file.write(f"weighted_average_correlation_val: {weighted_average_correlation_val:.2f}\n")
    print(f"weighted_average_r2_score_val: {weighted_average_r2_score_val:.2f}")
    output_file.write(f"weighted_average_r2_score_val: {weighted_average_r2_score_val:.2f}\n")
    print(f"weighted_average_correlation_test: {weighted_average_correlation_test:.2f}")
    output_file.write(f"weighted_average_correlation_test: {weighted_average_correlation_test:.2f}\n")
    print(f"weighted_average_r2_score_test: {weighted_average_r2_score_test:.2f}")
    output_file.write(f"weighted_average_r2_score_test: {weighted_average_r2_score_test:.2f}\n")

    bad_clusters_size_sum = 0
    total_clusters_size_sum = 0

    for cluster_size in cluster_sizes:
        total_clusters_size_sum += cluster_size
    
    for bad_cluster_index in bad_clusters_indices:
        bad_clusters_size_sum += len(clusters[bad_cluster_index])

    bad_clusters_rate = bad_clusters_size_sum / total_clusters_size_sum

    print(f"bad clusters rate: {bad_clusters_rate:.2f}")
    output_file.write(f"bad clusters rate: {bad_clusters_rate:.2f}\n")
    print(f"bad cluster indices: {bad_clusters_indices}")
    output_file.write(f"bad cluster indices: {bad_clusters_indices}\n")

In [16]:
def show_results(model, clusters, conv_width):
    cluster_sizes, average_correlations_val, average_r2_scores_val, average_correlations_test, average_r2_scores_test = do_investigations(model, clusters, conv_width)
    print_metrics(cluster_sizes, average_correlations_val, average_r2_scores_val, average_correlations_test, average_r2_scores_test, clusters)

In [17]:
CONV_WIDTH = 3

In [18]:
multi_step_dense_model = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])

In [19]:
def get_clusters(path):
    file = open(path, 'r')
    lines = file.readlines()
    file.close()

    clusters = []

    for line in lines:
        if line.strip() == '':
            continue
        clusters.append(convert_line_to_list(line))

    return clusters

In [38]:
first_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kmeans 5 0.1 20.txt"
second_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kmeans 15 0.1 20.txt"
third_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kmeans 30 0.1 20.txt"
fourth_path = r"C:\Coding\final-qualifying-work_2023-2024\ClusteringCpp\output\kmeans 60 0.1 20.txt"

output_file = open("best-model.txt", "w")

selected_path = fourth_path

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

print(f"msd {os.path.split(selected_path)[-1]}")
output_file.write(f"msd {os.path.split(selected_path)[-1]}\n")
show_results(multi_step_dense_model, get_clusters(selected_path), CONV_WIDTH)

print('------------------------------------------------------')
output_file.write('------------------------------------------------------\n')

output_file.close()

------------------------------------------------------
msd kmeans 60 0.1 20.txt
---do_investigations. cluster index : 0---


IntProgress(value=0, max=110)

Epoch 1/20
368/368 [==============================] - 3s 4ms/step - loss: 0.3090 - mean_absolute_error: 0.3613 - val_loss: 0.1612 - val_mean_absolute_error: 0.2766
Epoch 2/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2127 - mean_absolute_error: 0.3103 - val_loss: 0.1577 - val_mean_absolute_error: 0.2695
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2110 - mean_absolute_error: 0.3096 - val_loss: 0.1575 - val_mean_absolute_error: 0.2704
Epoch 4/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2108 - mean_absolute_error: 0.3085 - val_loss: 0.1579 - val_mean_absolute_error: 0.2795
Epoch 5/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2106 - mean_absolute_error: 0.3080 - val_loss: 0.1580 - val_mean_absolute_error: 0.2768
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.79
***VALIDATION***:
correlation related to cluster centre: 0.91
r2 score related to 

IntProgress(value=0, max=110)

INFO:tensorflow:Assets written to: out\saved-models\model for cluster 0\assets


...

Epoch 1/20
368/368 [==============================] - 2s 3ms/step - loss: 0.3142 - mean_absolute_error: 0.3324 - val_loss: 0.2774 - val_mean_absolute_error: 0.2838
Epoch 2/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2097 - mean_absolute_error: 0.2713 - val_loss: 0.2648 - val_mean_absolute_error: 0.2845
Epoch 3/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2094 - mean_absolute_error: 0.2710 - val_loss: 0.2842 - val_mean_absolute_error: 0.2990
Epoch 4/20
368/368 [==============================] - 1s 3ms/step - loss: 0.2097 - mean_absolute_error: 0.2700 - val_loss: 0.3176 - val_mean_absolute_error: 0.3180
***TRAIN***:
correlation related to cluster centre: 0.89
r2 score related to cluster centre: 0.77
***VALIDATION***:
correlation related to cluster centre: 0.90
r2 score related to cluster centre: 0.77
***TEST***:
correlation related to cluster centre: 0.88
r2 score related to cluster centre: 0.76


IntProgress(value=0, max=167)

INFO:tensorflow:Assets written to: out\saved-models\model for cluster 21\assets


INFO:tensorflow:Assets written to: out\saved-models\model for cluster 21\assets


***VALIDATION***:
average correlation val: 0.90
average r2 val: 0.76
***TEST***:
average correlation test: 0.88
average r2 test: 0.75
weighted_average_correlation_val: 0.90
weighted_average_r2_score_val: 0.79
weighted_average_correlation_test: 0.88
weighted_average_r2_score_test: 0.75
bad clusters rate: 0.50
bad cluster indices: [12, 20]
------------------------------------------------------
